<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [0]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 37.3MB/s]
  0% 0.00/948k [00:00<?, ?B/s]
100% 948k/948k [00:00<00:00, 61.4MB/s]
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 64.8MB/s]
  0% 0.00/3.81M [00:00<?, ?B/s]
100% 3.81M/3.81M [00:00<00:00, 123MB/s]


In [0]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip


Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime

In [0]:
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape, df_sample.shape


((59400, 39), (14358, 39), (59400, 2), (14358, 2))

In [0]:
def num_cleanup(X):
  '''
  date_recorded
  ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year']
  '''
  
  # categorical value nan converted to unknown to be processed by regression.
  X1 = X.fillna('unknown')
  
  # added recorded_year_built
  # convert date_recorded as year 
  dates = pd.to_dateime(X2.date_recorded)
  X2['recorded_year_built'] = dates.dt.year - X2['construction_year']
  X2.loc[X2['recorded_year_built' > 1000]] = 0
  X2.date_recorded = dates.dt.year
  
  #
  bins=[-100,0,100,500,1000,1250,1500,2000]
  labels=[0,100,500,1000,1250,1500,2000]
  X2['gps_height_bin']= pd.cut(X2.gps_height, bins, labels=labels)
  
  type_dict = {'amount_tsh':'float64',    # nunique()=98
               'date_recorded':'float64', # year
               'gps_height':'float64',    #bins
               'longitude':'float64',
               'latitude':'float64',
               'num_private':'float64',   # nunique()=65
               'population':'float64',    # nunique()=1049
               'construction_year':'float64',   # nunique()=55
               'recorded_year_built':'float64', #added
               'region_code':'float64',   # nunique()=27
               'district_code':'float64'  # nunique=20 }

In [0]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
df_train.select_dtypes('number').head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,,,,,,,,,
69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009
67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986
19728,0.0,0,31.130847,-1.825359,0,18,1,0,0
